In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from utils import *

In [2]:
event = "코나 화재"
communities = ['clien', 'bobae', 'fmkorea', 'naver_cafe']

In [3]:
per_community_dfs = []
for community in communities:
    posts_df = pd.read_csv(f'../data/{event}/{community}_posts.csv') 
    posts_df['from'] = community
    posts_df.created_at = posts_df.created_at.apply(parse_dates)

    comments_df = pd.read_csv(f'../data/{event}/{community}_comments.csv') 
    if community=='clien':
        comments_df.columns = ['cmt_author', 'cmt_count', 'post_id', 'cmt_created_at']
    comments_df = comments_df.groupby(['post_id'], as_index = False).agg({
        'cmt_author': ['count'],
    })

    comments_df.columns = comments_df.columns.droplevel(0)
    comments_df.columns = ['post_id', 'cmt_count']
    per_community_df = pd.merge(posts_df, comments_df, left_on='id', right_on='post_id', how='left')
    per_community_dfs.append(per_community_df)
df = pd.concat(per_community_dfs)

In [4]:

# naver_cafe_posts_df = pd.read_csv(f'../data/{event}/naver_cafe_posts.csv') 
# naver_cafe_posts_df['from'] = 'naver_cafe'
# naver_cafe_posts_df.created_at = naver_cafe_posts_df.created_at.apply(parse_dates)

# naver_cafe_comments_df = pd.read_csv(f'../data/{event}/naver_cafe_comments.csv') 
# naver_cafe_comments_df = naver_cafe_comments_df.groupby(['post_id'], as_index = False).agg({
#     'cmt_author': ['count'],
# })

# naver_cafe_comments_df.columns = naver_cafe_comments_df.columns.droplevel(0)
# naver_cafe_comments_df.columns = ['post_id', 'cmt_count']
# naver_cafe_df = pd.merge(naver_cafe_posts_df, naver_cafe_comments_df, left_on='id', right_on='post_id', how='left')

In [5]:
# clien_posts_df = pd.read_csv(f'../data/{event}/clien_posts.csv') 
# clien_posts_df['from'] = 'clien'
# clien_posts_df.created_at = clien_posts_df.created_at.apply(parse_dates)

# clien_comments_df = pd.read_csv(f'../data/{event}/clien_comments.csv') 
# clien_comments_df.columns = ['cmt_author', 'cmt_count', 'post_id', 'cmt_created_at'] # TODO: clien dataset cmt_author, post_id가 바뀌어 있음.

# clien_comments_df = clien_comments_df.groupby(['post_id'], as_index = False).agg({
#     'cmt_author': ['count'],
# })

# clien_comments_df.columns = clien_comments_df.columns.droplevel(0)
# clien_comments_df.columns = ['post_id', 'cmt_count']
# clien_df = pd.merge(clien_posts_df, clien_comments_df, left_on='id', right_on='post_id', how='left')

In [6]:
# bobae_posts_df = pd.read_csv(f'../data/{event}/bobae_posts.csv') 
# bobae_posts_df['from'] = 'bobae'
# bobae_posts_df.created_at = bobae_posts_df.created_at.apply(parse_dates)

# bobae_comments_df = pd.read_csv(f'../data/{event}/bobae_comments.csv') 
# bobae_comments_df = bobae_comments_df.groupby(['post_id'], as_index = False).agg({
#     'cmt_author': ['count'],
# })

# bobae_comments_df.columns = bobae_comments_df.columns.droplevel(0)
# bobae_comments_df.columns = ['post_id', 'cmt_count']
# bobae_df = pd.merge(bobae_posts_df, bobae_comments_df, left_on='id', right_on='post_id', how='left')

In [7]:
# fmkorea_posts_df = pd.read_csv(f'../data/{event}/fmkorea_posts.csv') 
# fmkorea_posts_df['from'] = 'fmkorea'
# fmkorea_posts_df.created_at = fmkorea_posts_df.created_at.apply(parse_dates)

# fmkorea_comments_df = pd.read_csv(f'../data/{event}/fmkorea_comments.csv') 
# fmkorea_comments_df = fmkorea_comments_df.groupby(['post_id'], as_index = False).agg({
#     'cmt_author': ['count'],
# })

# fmkorea_comments_df.columns = fmkorea_comments_df.columns.droplevel(0)
# fmkorea_comments_df.columns = ['post_id', 'cmt_count']
# fmkorea_df = pd.merge(fmkorea_posts_df, fmkorea_comments_df, left_on='id', right_on='post_id', how='left')

In [8]:
print(df.shape)
df.isnull().sum()

(3517, 12)


id               0
title            0
content        324
likes            6
url              0
author           0
views            0
created_at      15
updated_at    3271
from             0
post_id        184
cmt_count      184
dtype: int64

In [9]:
# Preprocessing
df = df.dropna(subset=['created_at']) # 생성 시간이 없는 게시물 제거
df.views = df.views.map(str).apply(remove_commna).apply(convert_str_to_int)
df.likes = df.likes.map(str).apply(remove_commna).apply(convert_str_to_float)
df.cmt_count = df.cmt_count.fillna(0).map(int)
print(df.shape)

(3502, 12)


In [10]:
# filtering by keyword & add created_day column
keywords = list(event.split())
filtered_df = filter_by_keyword(df, keywords)
filtered_df = add_created_day_col(filtered_df).sort_values(by=['created_at'])

In [11]:
real_events = [
    ("2020-10-08", "Red", "리콜 발표"), # 특정 기간(2017년 9월 29일부터 2020년 3월 13일)동안 생산된 코나 일렉트릭 25,564대를 리콜 
    ("2020-12-18", "Red", "단종"), # 코나 단종
    ("2021-03-22", "Red", "전량 리콜"), # 코나 전량 배터리 리콜 결정
    ("2023-02-28", "Red", "2세대 공개"), # 코나 EV 2세대 디자인 필름 공개
    ("2023-04-13", "Red", "2세대 출시"), # 코나 EV 2세대 정식 출시
    ("2021-02-24", "#79db93", "화재"), # 국토부에서 배터리 결함 발표 
    ("2020-04-02", "#79db93", "화재"), # 코나 화재 (경기도 안산)
    ("2020-05-29", "#79db93", "화재"), # 코나 화재 (대구 북구 주차장)
    ("2020-08-07", "#79db93", "화재"), # 코나 화재 (대구 북구 주차장)
    ("2020-08-15", "#79db93", "화재"), # 코나 화재 (전북 정읍시 주차장)
    ("2020-09-26", "#79db93", "화재"), # 코나 화재 (제주)
    ("2020-08-07", "#79db93", "화재"), # 코나 화재 (대구 북구 주차장)
    ("2020-10-04", "#79db93", "화재"), # 코나 화재 (대구 달성군 주차장)
    ("2020-10-17", "#79db93", "화재"), # 코나 화재 (남양주)
    ("2021-01-23", "#79db93", "화재"), # 코나 화재 (대구 달서구 충전소)
    ("2021-02-16", "#79db93", "화재"), # LG 배터리 사용한 현대 전기버스 화재
    ("2021-06-23", "#79db93", "화재"), # 코나 화재 (충남 보령)
    ("2021-07-01", "#79db93", "화재"), # 코나 화재 (세종시)
    ("2022-01-11", "#79db93", "화재"), # 코나 화재 (충남 태안읍)
    ("2022-02-12", "#79db93", "화재"), # 코나 화재 (부산 지하주차장)
    ("2023-05-24", "#79db93", "화재"), # 코나 화재 (대구 달성군)
]

In [12]:
filtered_df

,id,title,content,likes,url,author,views,created_at,updated_at,from,post_id,cmt_count,created_day
248,13761291,캐나다에서 코나EV 폭발 사고 발생,3월에 구매한 코나 EV가 차고에서 검은 연기가 피어오르더니 터졌다고 합니다.소방대...,1.0,https://www.clien.net/service/board/cm_car/137...,크로스경사,7761,2019-07-28 14:45:45,2019-07-28 14:47:12,clien,13761291.0,17,2019-07-28
1218,347353,배터리 화재방지에 좋은 습관,NaN,0.0,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,라바I니로I광주,493,2019-08-01 13:33:00,NaN,naver_cafe,347353.0,2,2019-08-01
1217,349133,중고차 살 때 꼭 확인 해야 하는 것이 있을까요?,NaN,0.0,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,슈페리어I코나I제주,84,2019-08-06 05:49:00,NaN,naver_cafe,NaN,0,2019-08-06
1216,351820,테슬라 모델 3 옵션 변경,NaN,0.0,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,뭉치I코나니로I설,1288,2019-08-13 08:30:00,NaN,naver_cafe,351820.0,6,2019-08-13
247,13817388,코나EV 화재 발생,오늘 (13일) 새벽에 아파트에서 충전중인 코나EV가 화재 나서 전소되었다고 하네요...,1.0,https://www.clien.net/service/board/cm_car/138...,솔로맥,4437,2019-08-13 09:16:45,2019-08-13 09:17:51,clien,13817388.0,9,2019-08-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1351,1539623,제 가 산 증인입니다 이 정도면 거의 포레스트 검프,-과거 a7 디젤 구입하고 몇개월 후 폭바 아우디 디젤게이트 터짐 시세 폭망-bmw...,3.0,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,깨굴I니로I인천,802,2024-08-14 13:46:00,NaN,naver_cafe,1539623.0,27,2024-08-14
1350,1540021,⭐전기차 모델별 배터리 안전 마진 총정리,"최근 서울시가 아파트 지하 주차장 진입 시 ""전기차 충전율 90% 제한"" 규정을 밝...",35.0,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,완속IEV3I경기,3287,2024-08-14 19:12:00,NaN,naver_cafe,1540021.0,39,2024-08-14
1341,1541556,전기차 화재 및 구입보조금 개선안 생각,NaN,0.0,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,크런치I코나I군포,150,2024-08-16 12:50:00,NaN,naver_cafe,1541556.0,1,2024-08-16
1340,1541660,"전기차 입문이 짧아, 전기차주 이미지가 이런걸까요?","최초 전기차에 관심이 가진건, 코나EV 구매시, 충전카드 주는 혜택을 보면서였는데....",0.0,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,와우l아이오닉5l광주,354,2024-08-16 13:53:00,NaN,naver_cafe,1541660.0,12,2024-08-16


In [13]:
# Per day Post (count)
vis_df = filtered_df
day_start = '2020-08-01'
day_end = '2021-12-31'
title = "Per day Number of Posts (count)"
result_df = plot_per_day_post_counts_with_events(vis_df, title=title, day_start=day_start, day_end=day_end)

In [14]:
# Per day number of views (total)
vis_df = filtered_df
day_start = '2020-08-01'
day_end = '2021-12-31'
title = "Per day Number of Views (Total)"
result_df = plot_per_day_target_val_sum_with_events(vis_df, title=title, target_val='views', day_start=day_start, day_end=day_end)

In [15]:
# Per day number of comments (total)
vis_df = filtered_df
day_start = '2020-08-01'
day_end = '2021-12-31'
title = "Per day Number of Comments (Total)"
result_df = plot_per_day_target_val_sum_with_events(vis_df, title=title, target_val='cmt_count', day_start=day_start, day_end=day_end)

In [16]:
# Per day number of likes (total)
vis_df = filtered_df
day_start = '2020-08-01'
day_end = '2021-12-31'
title = "Per day Number of Likes (Total)"
result_df = plot_per_day_target_val_sum_with_events(vis_df, title=title, target_val='likes', day_start=day_start, day_end=day_end)

In [17]:
a = filtered_df[filtered_df['created_day'] =='2020-11-05']